# Scenario Extraction

In [2]:
%load_ext autoreload
%autoreload 2
import os
import csv
import json
import matplotlib.pyplot as plt
from traffic_object import TrafficObject
from tools import dump_json, maybe_makedirs
from objects import Scenario

In [3]:
root_dir = '../' # This has to be the path to Problem folder (not the extraction folder)

In [7]:
for recording_id in range(1, 61):
    # make a copy of the template dict and fill it for current recording
    recording_dict = dict()
    recording_dict['recording_meta_file'] = os.path.join(root_dir, 'highd', '{recording_id}_recordingMeta.csv').format(recording_id=str(recording_id).zfill(2))
    recording_dict['tracks_meta_file'] = os.path.join(root_dir, 'highd', '{recording_id}_tracksMeta.csv').format(recording_id=str(recording_id).zfill(2))
    recording_dict['tracks_file'] = os.path.join(root_dir, 'highd', '{recording_id}_tracks.csv').format(recording_id=str(recording_id).zfill(2))

    # read the 3 required csv files for the current recording recording
    recording_meta_reader = csv.DictReader(open(recording_dict['recording_meta_file']))
    tracks_meta_reader = csv.DictReader(open(recording_dict['tracks_meta_file']))
    tracks_reader = csv.DictReader(open(recording_dict['tracks_file']))
    # read the only line from the meta file
    recording_meta_row = list(recording_meta_reader)[0]

    # create and init one traffic objects for each vehicle available in the recording
    num_vehicles = int(recording_meta_row["numVehicles"])
    to_dict = dict()
    for vehicle_id in range(1, num_vehicles + 1):
        to = TrafficObject(vehicle_id, tracks_reader, tracks_meta_reader, recording_meta_row)
        # ignore traffic objects which have no appearance after ignoring the starting frames
        if len(to.time_stamps) == 0:  # this may happen because of config.extraction["discard_first_n_frames"]
            continue
        to_dict[vehicle_id] = to   

    # set the calculated traffic object dict for each traffic object
    for vehicle_id in sorted(to_dict.keys()):
        to = to_dict[vehicle_id]
        to.set_others(to_dict)
    len(list(to.to_dict.keys())) 

    # determine the possible relevant other traffic objects for each traffic object (step 1)
    for vehicle_id in sorted(to_dict.keys()):
        to = to_dict[vehicle_id]
        to.determine_possible_relevant_others()
    to.possible_relevant_others

    # determine the relevant other traffic objects for each traffic object (step 2 and 3)
    for vehicle_id in sorted(to_dict.keys()):
        to = to_dict[vehicle_id]
        to.determine_relevant_others(include_8_car_filter=True)  # true = with 8 car filter, false = without 8 car filter
    to.time_stamps[0].relevant_others

    for vehicle_id in sorted(to_dict.keys()):
        to = to_dict[vehicle_id]
        to.extract_maneuvers()

    for vehicle_id in sorted(to_dict.keys()):
        to = to_dict[vehicle_id]
        to.extract_scenarios()

    scenarios = list()
    scenario_data = list()
    counter = 1  # counter for marking the scenarios
    for vehicle_id in sorted(to_dict.keys()):  # check every traffic object
        to = to_dict[vehicle_id]
        for s in to.scenario_list:  # check every lane change maneuver for this traffic object
            if len(s.traffic_objects_id) > 0:  # include scenario only if other traffic objects are involved

                # set the unique scenario number
                s.scenario_number = counter

                # appedn the scenario to the list for later meta data extraction 
                scenarios.append(s)

                # create file name
                file_name_data = os.path.join(os.getcwd(), "data_all", "scenario_" + str(recording_id).zfill(2) + "_" + str(s.scenario_number).zfill(3) + ".json")

                # dump to json
                df = s.evaluate(to_dict)
                df = df.reset_index()
                scenario_data.append(df)
                dump_json(file_name_data, json.loads(df.to_json(orient='records')))

                # update counter
                counter += 1

nario_60_085.json
Writing dump: /Users/kitty/GitRepos/RandomRepos/stanford-2019/extraction/data_all/scenario_60_086.json
Writing dump: /Users/kitty/GitRepos/RandomRepos/stanford-2019/extraction/data_all/scenario_60_087.json
Writing dump: /Users/kitty/GitRepos/RandomRepos/stanford-2019/extraction/data_all/scenario_60_088.json
Writing dump: /Users/kitty/GitRepos/RandomRepos/stanford-2019/extraction/data_all/scenario_60_089.json
Writing dump: /Users/kitty/GitRepos/RandomRepos/stanford-2019/extraction/data_all/scenario_60_090.json
Writing dump: /Users/kitty/GitRepos/RandomRepos/stanford-2019/extraction/data_all/scenario_60_091.json
Writing dump: /Users/kitty/GitRepos/RandomRepos/stanford-2019/extraction/data_all/scenario_60_092.json
Writing dump: /Users/kitty/GitRepos/RandomRepos/stanford-2019/extraction/data_all/scenario_60_093.json
Writing dump: /Users/kitty/GitRepos/RandomRepos/stanford-2019/extraction/data_all/scenario_60_094.json
Writing dump: /Users/kitty/GitRepos/RandomRepos/stanfor